# 텐서플로에서 데이터 적재와 전처리하기
* 텐서플로는 데이터 API덕분에  데이터 로드와 전처리를 쉽게한다.
* CSV,이진 파일, TFRecord포맷의 이진파일 가능
* sql데이터베이스에서 읽는 기능을 지원한다.
* 사용자 정의 전처리 층을 만들 수 있다. 표준 전처리 층도 있다.
  * TF.변환 -> 훈련 전에 전체 훈련 세트에 대해 실행하는 전처리 함수를 작성 할 수 있다.
  * TF데이터셋 -> 각종 데이터셋을 다운로드 할 수 있는 편리한 함수를 제공한다. 대용량 데이터셋도 포함된다.
---
### 데이터 API
* 데이터 API중심에는 데이터셋의 개념이 있다. 일반적으로 점진적으로 읽는 데이터셋 사용.
* tf.data.Dataset.from_tensor_slices()로 메모리에서 전체 데이터셋 생성


In [126]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sklearn
import sys

In [127]:
x= tf.range(10)
dataset=tf.data.Dataset.from_tensor_slices(x)
dataset
# 텐서를 받아 x의 각 원소가 아이템을 ㅗ표현되는 데이터셋을 만든다 즉 0~9의 10개의 아이탬 가짐.

<TensorSliceDataset shapes: (), types: tf.int32>

In [128]:
for item in dataset:
  print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


### 연쇄 변환
* 변환 메서드를 호출하여 여러 종류의 변환을 수행 가능
* 원본을 3번 반복하고 batch로 새로운 데이터셋이 만들어진다 7개씨 묶음.

In [129]:
dataset=dataset.repeat(3).batch(7,drop_remainder=True)# 모자란 마지막 배치 버림
for item in dataset:
  print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int32)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int32)


* map()메서드로 아이템 변환가능. 각 아이템을 변환하는 것

In [130]:
dataset=dataset.map(lambda x:x*2)
for item in dataset:
  print(item)

tf.Tensor([ 0  2  4  6  8 10 12], shape=(7,), dtype=int32)
tf.Tensor([14 16 18  0  2  4  6], shape=(7,), dtype=int32)
tf.Tensor([ 8 10 12 14 16 18  0], shape=(7,), dtype=int32)
tf.Tensor([ 2  4  6  8 10 12 14], shape=(7,), dtype=int32)


* aply메서드로 데이터셋 전체에 변환을 적용한다.
* unbatch 사용 하나의 정수텐서로 변환

In [131]:
dataset=dataset.apply(tf.data.experimental.unbatch())

In [132]:
for item in dataset:
  print(item)


tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(10, shape=(), dtype=int32)
tf.Tensor(12, shape=(), dtype=int32)
tf.Tensor(14, shape=(), dtype=int32)
tf.Tensor(16, shape=(), dtype=int32)
tf.Tensor(18, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(10, shape=(), dtype=int32)
tf.Tensor(12, shape=(), dtype=int32)
tf.Tensor(14, shape=(), dtype=int32)
tf.Tensor(16, shape=(), dtype=int32)
tf.Tensor(18, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(10, shape=(), dtype=int32)
tf.Tensor(12, shape=(), dtype=int32)
tf.Tensor(14, sh

In [133]:
dataset=dataset.filter(lambda x:x<10)
for item in dataset:
  print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)


* 데이터셋에 있는 몇 개의 아이템만 보려할때
* dataset.take()

In [134]:
for item in dataset.take(3):
  print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)


### 데이터 셔플링
* 경사하강법은 훈련 세트에 있는 샘플이 독립적이고 동일한 분포일때 최고의 성능을 발휘.(4강 참조)
* 이는 shuffle()메서드로 섞는것으로 가능하다.
* buffer_size 개수 만큼 추출하여 버퍼에 채워서 새로운 아이템을 버퍼에서 래넘하게 꺼내 반환하고 원본 데이터셋에서 새로운 아이탬을 추출해 체운다.
* 원본의 모든 아이템이 사용될 때까지 반복된다. 다음 버퍼가 비워질때가지 랜덤하게 아이템을 반환한다.
* 셔플링을 위해서 버퍼 크기를 충분히 크게 지정해야한다.


In [135]:
dataset=tf.data.Dataset.range(10).repeat(3)# 0-9까지 3번반복
dataset=dataset.shuffle(buffer_size=5,seed=42).batch(7)# 버퍼크기 5로 섞어서 7배치로 나눔
for item in dataset:
  print(item)

tf.Tensor([0 1 6 5 7 3 9], shape=(7,), dtype=int64)
tf.Tensor([8 2 1 0 4 6 4], shape=(7,), dtype=int64)
tf.Tensor([7 2 5 9 2 1 3], shape=(7,), dtype=int64)
tf.Tensor([4 3 8 7 9 5 0], shape=(7,), dtype=int64)
tf.Tensor([8 6], shape=(2,), dtype=int64)


* 대규모 데이터셋은 버퍼크기로는 적절하지 않을 수 있다.
* 따라서 데이터 자체를 미리 섞는것도 좋다.
* 섞여있더라도 에포크마다 한번 더 섞는다. 그렇지 않으면 에포크마다 동일한 순서가 반복되어 편향이 추가된다.

In [136]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
scaler.fit(X_train)
X_mean = scaler.mean_
X_std = scaler.scale_

In [137]:
def save_to_multiple_csv_files(data, name_prefix, header=None, n_parts=10):
    housing_dir = os.path.join("datasets", "housing")
    os.makedirs(housing_dir, exist_ok=True)
    path_format = os.path.join(housing_dir, "my_{}_{:02d}.csv")

    filepaths = []
    m = len(data)
    for file_idx, row_indices in enumerate(np.array_split(np.arange(m), n_parts)):
        part_csv = path_format.format(name_prefix, file_idx)
        filepaths.append(part_csv)
        with open(part_csv, "wt", encoding="utf-8") as f:
            if header is not None:
                f.write(header)
                f.write("\n")
            for row_idx in row_indices:
                f.write(",".join([repr(col) for col in data[row_idx]]))
                f.write("\n")
    return filepaths

In [138]:
train_data = np.c_[X_train, y_train]
valid_data = np.c_[X_valid, y_valid]
test_data = np.c_[X_test, y_test]
header_cols = housing.feature_names + ["MedianHouseValue"]
header = ",".join(header_cols)

train_filepaths = save_to_multiple_csv_files(train_data, "train", header, n_parts=20)
valid_filepaths = save_to_multiple_csv_files(valid_data, "valid", header, n_parts=10)
test_filepaths = save_to_multiple_csv_files(test_data, "test", header, n_parts=10)

In [139]:
import pandas as pd

pd.read_csv(train_filepaths[0]).head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedianHouseValue
0,3.5214,15.0,3.049945,1.106548,1447.0,1.605993,37.63,-122.43,1.442
1,5.3275,5.0,6.490060,0.991054,3464.0,3.443340,33.69,-117.39,1.687
2,3.1000,29.0,7.542373,1.591525,1328.0,2.250847,38.44,-122.98,1.621
3,7.1736,12.0,6.289003,0.997442,1054.0,2.695652,33.55,-117.70,2.621
4,2.0549,13.0,5.312457,1.085092,3297.0,2.244384,33.93,-116.93,0.956


* 여러 파일에서 한줄씩 번갈아가며 읽기.

In [140]:
with open(train_filepaths[0]) as f:
    for i in range(5):
        print(f.readline(), end="")

MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedianHouseValue
3.5214,15.0,3.0499445061043287,1.106548279689234,1447.0,1.6059933407325193,37.63,-122.43,1.442
5.3275,5.0,6.490059642147117,0.9910536779324056,3464.0,3.4433399602385686,33.69,-117.39,1.687
3.1,29.0,7.5423728813559325,1.5915254237288134,1328.0,2.2508474576271187,38.44,-122.98,1.621
7.1736,12.0,6.289002557544757,0.9974424552429667,1054.0,2.6956521739130435,33.55,-117.7,2.621


In [141]:
train_filepaths

['datasets/housing/my_train_00.csv',
 'datasets/housing/my_train_01.csv',
 'datasets/housing/my_train_02.csv',
 'datasets/housing/my_train_03.csv',
 'datasets/housing/my_train_04.csv',
 'datasets/housing/my_train_05.csv',
 'datasets/housing/my_train_06.csv',
 'datasets/housing/my_train_07.csv',
 'datasets/housing/my_train_08.csv',
 'datasets/housing/my_train_09.csv',
 'datasets/housing/my_train_10.csv',
 'datasets/housing/my_train_11.csv',
 'datasets/housing/my_train_12.csv',
 'datasets/housing/my_train_13.csv',
 'datasets/housing/my_train_14.csv',
 'datasets/housing/my_train_15.csv',
 'datasets/housing/my_train_16.csv',
 'datasets/housing/my_train_17.csv',
 'datasets/housing/my_train_18.csv',
 'datasets/housing/my_train_19.csv']

* 일반적으로 경로를 섞은 데이터셋을 반환한다.
* 섞기를 원하지 않으면 shuffle=False로 지정

In [142]:
filepath_dataset = tf.data.Dataset.list_files(train_filepaths, seed=42,shuffle=False)

* interleave메서드로 한번에 다섯개의 파일을 한줄 씩 번갈아 읽는다.
* filepath_dataset의 5개의  파일 경로에서 데이터를 읽는 데이터셋을 만든다.
* 인터리브 데이터셋을 반복문에 사용하면 텍스트라인 데이터셋을 순회한다.

In [143]:
n_readers=5
dataset= filepath_dataset.interleave(
    lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
    cycle_length=n_readers
)

In [144]:
for line in dataset.take(5):
  print(line.numpy())

b'3.5214,15.0,3.0499445061043287,1.106548279689234,1447.0,1.6059933407325193,37.63,-122.43,1.442'
b'3.6875,44.0,4.524475524475524,0.993006993006993,457.0,3.195804195804196,34.04,-118.15,1.625'
b'1.6571,34.0,4.454976303317536,1.0876777251184835,1358.0,3.2180094786729856,37.94,-122.35,1.052'
b'4.5909,16.0,5.475877192982456,1.0964912280701755,1357.0,2.9758771929824563,33.63,-117.71,2.418'
b'4.2708,45.0,5.121387283236994,0.953757225433526,492.0,2.8439306358381504,37.48,-122.19,2.67'


* 이는 csv파일의 첫번째 행에 해당하며 순서는 랜덤하다.
* 바이트 스트링이다 이를 파싱하고 스케일을 조정할 필요가있다.
---
### 데이터 전처리
* 데이터 전처리를 위한 간단한 함수를 만든다.

In [145]:
n_inputs = 8 # X_train.shape[-1]

@tf.function
def preprocess(line): # CSV 한 라인을 받아 파싱한다.,
    defs = [0.] * n_inputs + [tf.constant([], dtype=tf.float32)]
    fields = tf.io.decode_csv(line, record_defaults=defs)
    x = tf.stack(fields[:-1])
    y = tf.stack(fields[-1:])
    return (x - X_mean) / X_std, y #표준화

In [146]:
preprocess(b'4.2083,44.0,5.3232,0.9171,846.0,2.3370,37.47,-122.2,2.782')

(<tf.Tensor: shape=(8,), dtype=float32, numpy=
 array([ 0.16579157,  1.216324  , -0.05204565, -0.39215982, -0.5277444 ,
        -0.2633488 ,  0.8543046 , -1.3072058 ], dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([2.782], dtype=float32)>)

In [147]:
def csv_reader_dataset(filepaths,repeat=1,n_readers=5,
                       n_read_threads=None,shuffle_buffer_size=10000,
                       n_parse_threads=5,batch_size=32):
  dataset=tf.data.Dataset.list_files(filepaths).repeat(repeat)#섞은 경로를 반환
  dataset = dataset.interleave(
        lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
        cycle_length=n_readers, num_parallel_calls=n_read_threads) # 경로로부터 파일 읽어들임.
  dataset=dataset.shuffle(shuffle_buffer_size)
  dataset=dataset.map(preprocess,num_parallel_calls=n_parse_threads)
  return dataset.batch(batch_size).prefetch(1)

##### 프리패치
* prefetch(1)을 호출하면 데이터셋은 항상 한 배치가 미리 준비되도록 최선을 다한다.
* 훈련 알고리즘이 한 배치로 작업하는 동안 이 데이터셋이 동시에 다음 배치를 준비한다.
* 성능에 중요하다~

* 데이터셋이 메모리에 모두 들어갈 정도로 작으면 RAM에 모두 캐싱하는 catch() 메서드를 사용해서 훈련속도 향상.
* 일반적으로 전처리 한 후 셔플링,반복,배치,프리패치 전에 캐싱 수행한다.

---
### tf.keras와 데이터셋 사용하기
*

In [148]:
train_set=csv_reader_dataset(train_filepaths)
test_set=csv_reader_dataset(test_filepaths)
valid_set=csv_reader_dataset(valid_filepaths)

In [149]:
import tensorflow.keras as keras
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(1),
])
model.compile(loss="mse",optimizer=keras.optimizers.SGD(lr=1e-3))
model.fit(train_set,epochs=10,batch_size=5,validation_data=valid_set)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/10
363/363 [==============================] - 1s 2ms/step - loss: 1.6391 - val_loss: 0.9463
Epoch 2/10
363/363 [==============================] - 1s 2ms/step - loss: 0.7066 - val_loss: 0.6543
Epoch 3/10
363/363 [==============================] - 1s 3ms/step - loss: 0.6346 - val_loss: 0.6065
Epoch 4/10
363/363 [==============================] - 1s 2ms/step - loss: 0.5977 - val_loss: 0.5705
Epoch 5/10
363/363 [==============================] - 1s 3ms/step - loss: 0.5701 - val_loss: 0.5420
Epoch 6/10
363/363 [==============================] - 1s 2ms/step - loss: 0.5477 - val_loss: 0.5275
Epoch 7/10
363/363 [==============================] - 1s 2ms/step - loss: 0.5294 - val_loss: 0.4944
Epoch 8/10
363/363 [==============================] - 1s 3ms/step - loss: 0.5130 - val_loss: 0.4850
Epoch 9/10
363/363 [==============================] - 1s 2ms/step - loss: 0.4997 - val_loss: 0.4670
Epoch 10/10
363/363 [==============================] - 1s 2ms/step - loss: 0.4879 - val_loss: 0.4587

In [150]:
print(model.evaluate(test_set))
new_set=test_set.take(3).map(lambda x,y:x)
model.predict(new_set)[0]

162/162 [==============================] - 0s 2ms/step - loss: 0.4768
0.4767601788043976


array([2.628373], dtype=float32)

In [151]:
train_set

<PrefetchDataset shapes: ((None, 8), (None, 1)), types: (tf.float32, tf.float32)>

In [152]:
optimizer = keras.optimizers.Nadam(lr=0.01)
loss_fn = keras.losses.mean_squared_error

n_epochs = 5
batch_size = 32
n_steps_per_epoch = len(X_train) // batch_size
total_steps = n_epochs * n_steps_per_epoch
global_step = 0
for X_batch, y_batch in train_set.take(total_steps):
    global_step += 1
    print("\rGlobal step {}/{}".format(global_step, total_steps), end="")
    with tf.GradientTape() as tape:
        y_pred = model(X_batch)
        main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
        loss = tf.add_n([main_loss] + model.losses)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Global step 363/1810

## TFRecord 포맷
* 대용량 데이터를 저장하고 효율적으로 읽기 위해 텐서플로가 선호하는 포맷은 TFRecord이다.
* 크기가 다른 연속된 이진 레코드를 저장하는 단순 이진 포맷이다.
* 기본적으로 이는 파일을 하나씩 차례로 읽는다. num_parallel_reads를 지정해서 여러 파일에서 레코드를 번갈아 읽을 수 있다.
* 앞의 list_files()와 interleave()를 사용해서 동일한 결과를 얻을 수 있다.
---

In [153]:
with tf.io.TFRecordWriter("my_data.tfrecord") as f:
  f.write(b"This is the first record")
  f.write(b"And this is the second record")

In [154]:
filepaths=["my_data.tfrecord"]
dataset=tf.data.TFRecordDataset(filepaths)
for item in dataset:
  print(item)

tf.Tensor(b'This is the first record', shape=(), dtype=string)
tf.Tensor(b'And this is the second record', shape=(), dtype=string)


In [155]:
filepaths = ["my_test_{}.tfrecord".format(i) for i in range(5)]
for i, filepath in enumerate(filepaths):
    with tf.io.TFRecordWriter(filepath) as f:
        for j in range(3):
            f.write("File {} record {}".format(i, j).encode("utf-8"))

dataset = tf.data.TFRecordDataset(filepaths, num_parallel_reads=3)
for item in dataset:
    print(item)

tf.Tensor(b'File 0 record 0', shape=(), dtype=string)
tf.Tensor(b'File 1 record 0', shape=(), dtype=string)
tf.Tensor(b'File 2 record 0', shape=(), dtype=string)
tf.Tensor(b'File 0 record 1', shape=(), dtype=string)
tf.Tensor(b'File 1 record 1', shape=(), dtype=string)
tf.Tensor(b'File 2 record 1', shape=(), dtype=string)
tf.Tensor(b'File 0 record 2', shape=(), dtype=string)
tf.Tensor(b'File 1 record 2', shape=(), dtype=string)
tf.Tensor(b'File 2 record 2', shape=(), dtype=string)
tf.Tensor(b'File 3 record 0', shape=(), dtype=string)
tf.Tensor(b'File 4 record 0', shape=(), dtype=string)
tf.Tensor(b'File 3 record 1', shape=(), dtype=string)
tf.Tensor(b'File 4 record 1', shape=(), dtype=string)
tf.Tensor(b'File 3 record 2', shape=(), dtype=string)
tf.Tensor(b'File 4 record 2', shape=(), dtype=string)


#### 압축된 TFRecord 파일
* options매개변수를 사용해서 압축된 TFRecord파일을 만들 수 있다.

In [156]:
options = tf.io.TFRecordOptions(compression_type="GZIP")
with tf.io.TFRecordWriter("my_compressed.tfrecord", options) as f:
    f.write(b"This is the first record")
    f.write(b"And this is the second record")

In [157]:
dataset = tf.data.TFRecordDataset(["my_compressed.tfrecord"],
                                  compression_type="GZIP")
for item in dataset:
    print(item)

tf.Tensor(b'This is the first record', shape=(), dtype=string)
tf.Tensor(b'And this is the second record', shape=(), dtype=string)


#### 프로토콜 버퍼 개요
* TFRecord는 직렬화된 프로토콜 버퍼를 담고있다.

In [158]:

%%writefile person.proto
syntax = "proto3"; #프로토콜 버퍼의 버전 3을 사용
message Person {
  string name = 1;#필드의 식별자.
  int32 id = 2;
  repeated string email = 3;
}

Overwriting person.proto


## 입력 트겅 전처리
* 신경망을 위해 모든 특성을 수치 특성으로 변환하고 정규화 해야한다.
* 범주형,텍스트를 숫자로 해야한다.
* Lambda층으로 표준화 수행

In [159]:
means=np.mean(X_train,axis=0,keepdims=True)
stds=np.std(X_train,axis=0,keepdims=True)
eps=keras.backend.epsilon()
model=keras.models.Sequential([
                               keras.layers.Lambda(lambda inputs: (inputs-means)/(stds+eps)),
                               keras.layers.Dense(30)
])

In [159]:
X_train